In [ ]:
!pip install opencv-python numpy

In [ ]:
import os
import cv2
import numpy as np

class FormFieldDetector:
    def __init__(self, output_dir='detected_fields'):
        """
        Initialize the form field detector with configurable parameters.

        Args:
            output_dir (str): Directory to save detected field images
        """
        # Configuration parameters
        self.MIN_LINE_LENGTH = 70
        self.MAX_LINE_GAP = 10
        self.FIELD_HEIGHT = 35  # Height of the box around detected lines
        self.LINE_BOTTOM_RATIO = 0.7  # Position of line from top of box (0.7 means 70% from top)

        # Create output directory if it doesn't exist
        self.output_dir = output_dir
        os.makedirs(self.output_dir, exist_ok=True)

    def detect_main_box(self, image):
        """Detect the main box containing personal information using contour detection."""
        # Get image dimensions
        height, width = image.shape[:2]

        # Focus on the top portion of the image where the main box typically is
        top_portion = int(height * 0.4)
        roi = image[:top_portion, :]

        # Convert to grayscale and apply adaptive thresholding
        if len(roi.shape) == 3:
            gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        else:
            gray = roi.copy()

        thresh = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
        )

        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Initialize variables for best box
        best_box = None
        max_area = 0
        target_ratio = 3.5  # Expected width/height ratio for the main box

        for contour in contours:
            # Approximate the contour to a polygon
            approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)

            # Check if it's a rectangle (4 corners)
            if len(approx) == 4:
                x, y, w, h = cv2.boundingRect(approx)
                area = w * h
                ratio = w / h if h != 0 else 0

                if (area > max_area and  # Largest area
                    y < top_portion * 0.8 and  # Near the top
                    ratio > 2 and ratio < 5 and  # Reasonable aspect ratio
                    w > width * 0.5):  # At least half the image width

                    max_area = area
                    best_box = approx

        if best_box is not None:
            return best_box.reshape((-1, 1, 2))

        return None

    def detect_field_regions(self, image, main_box, save_images=True, prefix='field'):
        """
        Detect field regions within the main box and optionally save them as images.

        Args:
            image (numpy.ndarray): Input image
            main_box (numpy.ndarray): Detected main box contour
            save_images (bool): Whether to save detected field images
            prefix (str): Prefix for saved image filenames

        Returns:
            list: Detected field regions with coordinates and optional image paths
        """
        # If no main box is detected, return empty list
        if main_box is None:
            print("No main box detected")
            return []

        # Get box boundaries
        x, y, w, h = cv2.boundingRect(main_box)

        # Extract only the main box region
        roi = image[y:y+h, x:x+w]

        # Convert to grayscale if needed
        if len(roi.shape) == 3:
            roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        else:
            roi_gray = roi

        # Apply thresholding to isolate lines
        thresh = cv2.adaptiveThreshold(
            roi_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
        )

        # Use morphological operations to enhance horizontal lines
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 1))
        horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel)

        # Find contours of horizontal lines
        contours, _ = cv2.findContours(horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter and sort contours
        valid_contours = []
        for contour in contours:
            x_line, y_line, w_line, h_line = cv2.boundingRect(contour)

            # Filter based on line width and position
            if w_line > self.MIN_LINE_LENGTH and w_line < w/2 and h_line < 20:
                valid_contours.append((y_line, contour))

        # Sort by vertical position
        valid_contours.sort(key=lambda x: x[0])

        # Create regions around each line
        detected_regions = []
        for i, (y_line, contour) in enumerate(valid_contours):
            x_line, y_line, w_line, h_line = cv2.boundingRect(contour)

            # Create a box around the line
            box_height = self.FIELD_HEIGHT

            # Calculate y position so the line is towards the bottom of the box
            y_start = max(0, y_line - int(box_height * self.LINE_BOTTOM_RATIO))

            # Define region coordinates (global coordinates)
            region_coords = (
                x + x_line,  # Global x coordinate
                y + y_start,  # Global y coordinate
                x + x_line + w_line + 30,  # Global x2 coordinate
                y + y_start + box_height  # Global y2 coordinate
            )

            # Extract region
            x1, y1, x2, y2 = region_coords
            region_img = image[y1:y2, x1:x2]

            # Prepare region data
            region_data = {
                'name': f'{prefix}_{i+1}',
                'coords': region_coords
            }

            # Save image if requested
            if save_images and region_img.size > 0:
                filename = os.path.join(
                    self.output_dir,
                    f'{prefix}_{i+1}.png'
                )
                cv2.imwrite(filename, region_img)
                region_data['image_path'] = filename

            detected_regions.append(region_data)

        return detected_regions

def process_images_in_directory(input_dir='.', output_dir='detected_fields3', image_extensions=['.jpg', '.jpeg', '.png', '.tif', '.tiff', '.bmp']):
    """
    Process all images in a specified directory.

    Args:
        input_dir (str): Directory containing images to process. Defaults to current directory.
        output_dir (str): Directory to save detected field images.
        image_extensions (list): List of valid image file extensions to process.

    Returns:
        dict: A dictionary with image filenames and their processing results
    """
    # Initialize the FormFieldDetector
    detector = FormFieldDetector(output_dir=output_dir)

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Results dictionary to track processing
    processing_results = {}

    # Iterate through files in the directory
    for filename in os.listdir(input_dir):
        # Check if file has a valid image extension
        if any(filename.lower().endswith(ext) for ext in image_extensions):
            # Full path to the image
            image_path = os.path.join(input_dir, filename)

            try:
                # Read the image
                image = cv2.imread(image_path)

                # Check if image was successfully read
                if image is None:
                    print(f"Could not read image: {filename}")
                    processing_results[filename] = "Error: Could not read image"
                    continue

                # Create a unique subdirectory for this image's fields
                # image_output_dir = os.path.join(output_dir, os.path.splitext(filename)[0])
                # os.makedirs(image_output_dir, exist_ok=True)

                # # Update detector's output directory
                # detector.output_dir = image_output_dir

                # Detect the main box
                main_box = detector.detect_main_box(image)

                if main_box is None:
                    print(f"No main box detected in {filename}")
                    processing_results[filename] = "Error: No main box detected"
                    continue

                # Detect and save field regions within the main box
                detected_regions = detector.detect_field_regions(
                    image,
                    main_box,
                    save_images=True,
                    prefix=os.path.splitext(filename)[0]
                )

                # Store processing results
                processing_results[filename] = {
                    'num_regions': len(detected_regions),
                    'regions': [region['name'] for region in detected_regions]
                }

                print(f"Processed {filename}: Detected {len(detected_regions)} regions")

            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                processing_results[filename] = f"Error: {str(e)}"

    return processing_results

def main():
    # Process images in the current directory
    results = process_images_in_directory()

    # Optionally, print a summary of processing
    print("\nProcessing Summary:")
    for filename, result in results.items():
        if isinstance(result, dict):
            print(f"{filename}: {result['num_regions']} regions detected")
        else:
            print(f"{filename}: {result}")

if __name__ == "__main__":
    main()

Processed 1000020614.jpg: Detected 16 regions
Processed 1000020603.jpg: Detected 14 regions
Processed 1000020569.jpg: Detected 14 regions
No main box detected in 1000020579.jpg
Processed 1000020607.jpg: Detected 15 regions
Processed 1000020613.jpg: Detected 14 regions
Processed 1000020615.jpg: Detected 15 regions
Processed 1000020602.jpg: Detected 14 regions
Processed 1000020577.jpg: Detected 18 regions
Processed 1000020585.jpg: Detected 14 regions
Processed 1000020593.jpg: Detected 15 regions
Processed 1000020587.jpg: Detected 15 regions
Processed 1000020589.jpg: Detected 14 regions
Processed 1000020600.jpg: Detected 14 regions
Processed 1000020572.jpg: Detected 14 regions
Processed 1000020605.jpg: Detected 14 regions
Processed 1000020601.jpg: Detected 14 regions
Processed 1000020599.jpg: Detected 17 regions
Processed 1000020612.jpg: Detected 14 regions
Processed 1000020608.jpg: Detected 14 regions
Processed 1000020592.jpg: Detected 15 regions
Processed 1000020571.jpg: Detected 14 reg

In [ ]:
import shutil

shutil.make_archive('detected_fields3', 'zip', 'detected_fields3')

'/content/detected_fields3.zip'

In [ ]:
from google.colab import files

files.download('/content/detected_fields3.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>